# EP

### But :  Réserver plusieurs noeuds d'un cluster et lancer smpirun ep à l'aide de "execo.action.Remote" 


#### Préparation de l'env.

In [1]:
%%bash -s
pip3 install --user execo


In [2]:
import execo
from execo import *
from execo_g5k import *
from execo_engine import *
import os
import sys
import json
import time
import re
from shutil import copy
from subprocess import check_output

Réservation noeuds

In [3]:
# Nom du job
jobname = ' '
# Nombre de noeuds
nodecount = 5
# Temps réservation
walltime = "6:0:0" 

# filters out Nancy's graphene cluster (buggy)
resources_selection = "-p \"cluster != 'graphene'\"" 

# Nancy
site = "nancy"

In [4]:
jobs = get_current_oar_jobs()
jobid = None
waiting_jobs = []
while jobs:
    j, site = jobs.pop()
    info = get_oar_job_info(j, site)
    if info['name'] == jobname:
        if info['state'] == 'Running':
            jobid = j
            print("A {} job is already running, using it. jobid is {}".format(jobname, jobid))
            break
        else:
            waiting_jobs.append(j)
if not jobid and not waiting_jobs:
    jobspec = OarSubmission(resources="/cluster=1/nodes={}".format(nodecount), walltime=walltime,
                            additional_options=resources_selection, job_type="deploy", name=jobname)
    jobid, _ = oarsub([(jobspec, site)]).pop()
    print("New job submitted, jobid is {}".format(jobid))
elif not jobid:
    print("One or more {} jobs exist ({}) but are not running.\n"
          " Connect to the frontend to see what is happening, and/or run the cell again.".format(
          jobname, ", ".join([str(j) for j in waiting_jobs])))

New job submitted, jobid is 1566428


On les range

In [57]:
nodes = get_oar_job_nodes(jobid)
nodes.sort(key=lambda n: n.address)
nodes

[Host('grisou-20.nancy.grid5000.fr'),
 Host('grisou-3.nancy.grid5000.fr'),
 Host('grisou-38.nancy.grid5000.fr'),
 Host('grisou-43.nancy.grid5000.fr'),
 Host('grisou-9.nancy.grid5000.fr')]

In [16]:
force_redeploy = False # set to True to force redeploying the OS on the nodes in the deployment section
environment_dsc_file = '.)/debian9-x64-bigdata-tutorial.yaml' # filename of the kadeploy environment file (YAML)

In [17]:
deployment = Deployment(hosts=nodes, env_file = os.path.abspath(environment_dsc_file),
                        other_options="-r ext4 --no-debug-mode")

deploy_ok, deploy_failed = deploy(deployment, check_deployed_command=not force_redeploy,
                              stdout_handlers=[sys.stdout],
                              stderr_handlers=[sys.stderr])

nancy: Deployment #D-040ff606-3540-4948-8521-a6812d491bac started
nancy: Grab the key file /home/nezzine/.ssh/authorized_keys
nancy: Grab the tarball file http://public.nancy.grid5000.fr/~pneyron/debian9-x64-bigdata-tutorial.tar.gz
nancy: Grab the postinstall file server:///grid5000/postinstalls/g5k-postinstall.tgz
nancy: Launching a deployment on grisou-[3,9,20,38,43].nancy.grid5000.fr
nancy: Performing a Deploy[SetDeploymentEnvUntrusted] step
nancy:   switch_pxe
nancy:   reboot
nancy:    * Performing a soft reboot on grisou-[3,9,20,38,43].nancy.grid5000.fr
nancy:   wait_reboot
nancy:   send_key_in_deploy_env
nancy:   create_partition_table
nancy:   format_deploy_part
nancy:   mount_deploy_part
nancy:   format_tmp_part
nancy:   format_swap_part
nancy: End of step Deploy[SetDeploymentEnvUntrusted] after 165s
nancy: Performing a Deploy[BroadcastEnvKascade] step
nancy:   send_environment
nancy:    * Broadcast time: 70s
nancy:   manage_admin_post_install
nancy:   manage_user_post_install


nancy: Warning: Permanently added 'nancy.grid5000.fr,172.16.79.101' (ECDSA) to the list of known hosts.
nancy: Connection to nancy.grid5000.fr closed.


In [8]:
# execo.action.Remote(cmd,hosts,connection_params,process_args)
Remote_test = execo.action.Remote(cmd='ps',hosts=nodes,connection_params=None, process_args=None)

In [9]:
Remote_test.run()

2018-05-18 09:27:12,258 WARNING: terminated: <SshProcess('ps', Host('grisou-20.nancy.grid5000.fr'), name=ps, started=True, start_date=2018-05-18 09:27:12+02:00, ended=True, end_date=2018-05-18 09:27:12+02:00, killed=False, error=False, error_reason=None, timeouted=False, expect_fail=False, write_error=False, exit_code=65280, ok=False, pid=5005, real cmd=('ssh', '-tt', '-o', 'BatchMode=yes', '-o', 'PasswordAuthentication=no', '-o', 'StrictHostKeyChecking=no', '-o', 'UserKnownHostsFile=/dev/null', '-o', 'ConnectTimeout=20', 'grisou-20.nancy.grid5000.fr', 'ps'))>
stdout:

stderr:
Authentication failed.

2018-05-18 09:27:12,283 WARNING: terminated: <SshProcess('ps', Host('grisou-3.nancy.grid5000.fr'), name=ps, started=True, start_date=2018-05-18 09:27:12+02:00, ended=True, end_date=2018-05-18 09:27:12+02:00, killed=False, error=False, error_reason=None, timeouted=False, expect_fail=False, write_error=False, exit_code=65280, ok=False, pid=5006, real cmd=('ssh', '-tt', '-o', 'BatchMode=yes',

Remote('ps', [Host('grisou-20.nancy.grid5000.fr'), Host('grisou-3.nancy.grid5000.fr'), Host('grisou-38.nancy.grid5000.fr'), Host('grisou-43.nancy.grid5000.fr'), Host('grisou-9.nancy.grid5000.fr')])

In [19]:
Remote_install = execo.action.Remote(cmd='apt-get install g++ libboost-all-dev && wget http://gforge.inria.fr/frs/download.php/latestfile/8/SimGrid-3.18.tar.gz && tar -xvf SimGrid-3.18.tar.gz && cd SimGrid-3.18 && cmake -DCMAKE_INSTALL_PREFIX=/usr/local -Denable_smpi=on -Denable_documentation=off && make -j && make check && make install -j',hosts=nodes,connection_params={'user':'root'})

In [20]:
Remote_install.run().ok

True

In [24]:
Remote_smpi_env = execo.action.Put(hosts=nodes,local_files=["griffon.xml","griffon_hostfile.txt","ep.c","nas_common.h","nas_common.c","Makefile"],connection_params={'user':'root'})
Remote_smpi_env.run().ok

True

In [25]:
Remote_smpi = execo.action.Remote(cmd ='make && make run',hosts=nodes,connection_params={'user':'root'})
Remote_smpi.run().ok

True

In [65]:
for i in range(0,nodecount) :
    execo.action.Get(hosts=nodes[i],remote_files=["smpi_results.txt"], local_location='./result/result_'+get_host_shortname(nodes[i])+'.txt',connection_params={'user':'root'}).run().ok

In [66]:
execo.Process("ls").run().stdout

'CMakeCache.txt\ndebian9-x64-bigdata-tutorial.yaml\nenv.sh\nEP\nep.c\nEP.ipynb\nep.o\ngriffon_hostfile.txt\ngriffon.xml\nMakefile\nnas_common.c\nnas_common.h\nnas_common.o\npublic\nresult\nresult_grisou-20.txt\nresult_grisou-38.txt\nresult_grisou-3.txt\nresult_grisou-43.txt\nsmpi_results.txt\ntraining\n'